In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/cassano.f/finetuning-harness/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
name = "bigcode/starcoder2-7b_base"
model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained(name)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.36s/it]


In [25]:
def make_prompt(ins, code, title="Changes to exercise1"):
    header = f"""<pr>Title: {title}
username_0: This PR resolves the following request.
{ins}<pr_status>opened<repo_name>code-editing/python-exercises<pr_base><pr_file>/problems/exercise1.py<pr_base_code>"""
    base_code = f"""<pr_base_code>{code}<pr_diff><pr_file>/problems/exercise1.py<pr_diff_hunk>"""
    prompt = header + base_code
    return prompt

code = """def add(x, y):
    return x + y"""

prompt = make_prompt("Add a function called `substract` that subtracts two numbers", code)
toks = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

In [26]:
import datasets
ds = datasets.load_dataset("nuprl/CanItEdit", split="test")

In [144]:
# 43 is two hunks
ex = ds[25]
before = ex["before"]
ins = ex["instruction_descriptive"]
prompt = make_prompt(ins, before)
toks = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

In [145]:
outs = model.generate(toks, max_new_tokens=2000, temperature=0, top_p=0.95, do_sample=False)[0]
print(len(outs))
print(len(toks))
outs = outs[len(toks[0]):]
print(len(outs))
dec = tokenizer.decode(outs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


2407
1
2000


In [146]:
pr_end_toks = [
    "<pr_review>", "<pr_comment>", "<pr>", "<pr_review_comment>"
]
# <pr_review>, <pr_comment>, <pr>


In [152]:
for tok in pr_end_toks:
    found = dec.find(tok)
    if found != -1:
        dec = dec[:found]
print(dec)

@@ -1,27 +1,32 @@
 from typing import List
 
 class Node:
     '''Simple node (No duplicate edges between nodes)'''
-    def __init__(self, id: int, out_edges: List[int]):
-        uniques = {}
-        for edge in out_edges:
-            if edge in uniques.keys():
-                raise RuntimeError
-            else:
-                uniques[edge] = True
-        self.id = id
-        self.in_edges = out_edges
+    def __init__(self, id: int, out_edges: List[int]):        
+        self.id = id
+        self.out_edges = out_edges
+        self.in_edges = []
+        self.visited = False
+        self.finished = False
+
+    def __repr__(self):
+        return f"Node({self.id}, {self.out_edges})"
 
 class Graph:
     '''Simple directed graph (No duplicate edges between nodes, no duplicate nodes)'''
-    def __init__(self, nodes: List[Node]):
-        uniques = {}
-        for node in nodes:
-            if node in uniques:
-                raise RuntimeError
-            else:
-      

In [148]:
hunks = dec.split("<pr_diff_hunk>")

In [149]:
for hunk in hunks:
    print(hunk)

@@ -1,27 +1,32 @@
 from typing import List
 
 class Node:
     '''Simple node (No duplicate edges between nodes)'''
-    def __init__(self, id: int, out_edges: List[int]):
-        uniques = {}
-        for edge in out_edges:
-            if edge in uniques.keys():
-                raise RuntimeError
-            else:
-                uniques[edge] = True
-        self.id = id
-        self.in_edges = out_edges
+    def __init__(self, id: int, out_edges: List[int]):        
+        self.id = id
+        self.out_edges = out_edges
+        self.in_edges = []
+        self.visited = False
+        self.finished = False
+
+    def __repr__(self):
+        return f"Node({self.id}, {self.out_edges})"
 
 class Graph:
     '''Simple directed graph (No duplicate edges between nodes, no duplicate nodes)'''
-    def __init__(self, nodes: List[Node]):
-        uniques = {}
-        for node in nodes:
-            if node in uniques:
-                raise RuntimeError
-            else:
-      

In [150]:
print(before)

from typing import List

class Node:
    '''Simple node (No duplicate edges between nodes)'''
    def __init__(self, id: int, out_edges: List[int]):
        uniques = {}
        for edge in out_edges:
            if edge in uniques.keys():
                raise RuntimeError
            else:
                uniques[edge] = True
        self.id = id
        self.in_edges = out_edges

class Graph:
    '''Simple directed graph (No duplicate edges between nodes, no duplicate nodes)'''
    def __init__(self, nodes: List[Node]):
        uniques = {}
        for node in nodes:
            if node in uniques:
                raise RuntimeError
            else:
                uniques[node] = True
        self.nodes = nodes
        
    def find_node(self, id: int):
        for node in self.nodes:
            if node.id == id:
                return node
    
    def topological_sort(self) -> List[Node]:
        return self.nodes
